In [5]:
# 问题来自李永乐老师的文章，https://weibo.com/ttarticle/p/show?id=2309404827450235551926
# 四只鸭子在一个圆形的水池里，每只鸭子的位置都是随机的。请问这四只鸭子在同一个半圆里的概率有多大？
# 通用解，包括>180度的解：通过求不可能的概率，即N个鸭子不可能在D角度的概率T。然后Ans=1-T
# 不存在一个角度D可以包括所有鸭子，就说明相邻鸭子之间的角度必须小于C=360-D
# 任意取一只鸭子作为0点，相邻鸭子间的距离为x_1，x_2，x_3，。。。x_{N-1}。
# 那么可以得到N个约束条件 0 <= x_i <= C, 360 - C <= sum(x_i) <= 360,这N个约束条件可以构成一个闭合的凸包 X
# 考虑全部的取值空间，即N只鸭子在圆上分布的所有可能性，会得到 0 <= a_i <= 360, sum(a_i) <= 360，同样这N个条件也可以构成一个闭合的凸包 A
# 不可能的概率T = X的体积/A的体积
# 为了简化，将不等式全部除以360
# A的体积很好求，是1/(n-1)!

# 问题在于如何求出X的体积
# 这时，我们把通过不等式两侧同时除C把问题标准化，x_i是服从于[0,1]均匀分布的样本，p = C/360。
# O <= x_i <= 1, (1-p)/p <= sum(x_i) <= 1/p
# sum(x_i) 服从于 Irwin-Hasll distribution, cdf(n-1,p)
# 所以X的体积是p^(n-1) * (cdf(n-1, 1/p) - cdf(n-1,(1-p)/p))

# 最终答案是 1 - p^(n-1) * (cdf(n-1, 1/p) - cdf(n-1,(1-p)/p)) / (n-1)!



import numpy as np
import math
from numpy import random
from fractions import Fraction

In [6]:
def sample(n):
    return random.rand(n)


def testOne(n,p):
    x = sample(n)
    x.sort()
    if x[0] > p or 1 - x[-1] > p:
        return False
    for i in range(1,n):
        if x[i] - x[i-1] > p:
            return False
    return True


def testT(n,p,t):
    r = 0
    for i in range(t):
        if testOne(n,p):
            r += 1
    return (r/t, r, t)

def testDuckT(n,degree,t):
    p, r, _= testT(n-1, 1 - degree / 360, t)
    return 1 - p, t - r, t

print(testDuckT(4,190,100000))

(0.58633, 58633, 100000)


In [16]:
# Irwin-Hasll distribution

def F(n):
    return math.factorial(n)

def C(n,m):
    ret = 1
    for i in range(m):
        ret *= (n - i)
    for i in range(1, m+1):
        ret //= i
    return ret

def cdf(n,x):
    int_x = int(x)
    ret = 0
    for k in range(int_x+1):
        #print(k,((-1)**k),C(n,k),((x - k)**n),((-1)**k) * C(n,k) * ((x - k)**n))
        ret += ((-1)**k) * C(n,k) * ((x - k)**n)
    return ret / F(n)

def no_sol_prob(n,p,debug=False):
    # n * p < 1
    # n < 1 / p
    a = cdf(n, 1 / p)
    if debug:
        print("a", n, 1/p, cdf(n,1/p), a)
    # n * p > 1 - p
    # n > (1 - p) / p
    b = cdf(n, (1 - p) / p)
    if debug:
        print("b", n, (1-p)/p, cdf(n,(1-p)/p), b)
        print("a-b",a-b)
    return a - b

def no_sol_prob_scaled(n, _p,debug=False):
    p = Fraction(_p)
    region_all = Fraction(1,F(n))
    if debug:
        print("region_all", region_all)
    region_p = p**n
    if debug:
        print("region_p", p, n, region_p)
        print("p/all", region_p / region_all)
    no_sol_region = no_sol_prob(n, p) * region_p
    if debug:
        print("no_sol_region", no_sol_prob(n,p,True), no_sol_region)
    return no_sol_region / region_all

def sol_prob(n, p):
    return 1 - no_sol_prob_scaled(n, p)

def duck_circle(n, degree, pp=True):
    ans = sol_prob(n-1, Fraction(360-degree,360))
    if pp:
        print("the probability of %d ducks being within degree %d is %s, %s" % (n, degree, ans, float(ans)))
    return n, degree, ans, float(ans)

duck_circle(4,190)

duck_n = 4
step = 5

for i in range(1, 360 // 5):
    print(duck_circle(4, i * step, False))
    
for i in range(1, 360 // 5):
    print(duck_circle(3, i * step, False))
    

the probability of 4 ducks being within degree 190 is 6847/11664, 0.587019890260631
(4, 5, Fraction(1, 93312), 1.071673525377229e-05)
(4, 10, Fraction(1, 11664), 8.573388203017832e-05)
(4, 15, Fraction(1, 3456), 0.00028935185185185184)
(4, 20, Fraction(1, 1458), 0.0006858710562414266)
(4, 25, Fraction(125, 93312), 0.0013395919067215364)
(4, 30, Fraction(1, 432), 0.0023148148148148147)
(4, 35, Fraction(343, 93312), 0.0036758401920438956)
(4, 40, Fraction(4, 729), 0.0054869684499314125)
(4, 45, Fraction(1, 128), 0.0078125)
(4, 50, Fraction(125, 11664), 0.010716735253772291)
(4, 55, Fraction(1331, 93312), 0.014263974622770919)
(4, 60, Fraction(1, 54), 0.018518518518518517)
(4, 65, Fraction(2197, 93312), 0.023544667352537723)
(4, 70, Fraction(343, 11664), 0.029406721536351165)
(4, 75, Fraction(125, 3456), 0.03616898148148148)
(4, 80, Fraction(32, 729), 0.0438957475994513)
(4, 85, Fraction(4913, 93312), 0.052651320301783266)
(4, 90, Fraction(1, 16), 0.0625)
(4, 95, Fraction(6859, 93312), 0.

In [18]:
def fast_no_sol_prob_scaled(n, _p, debug=False):
    p = Fraction(_p)
    int_x = int(1 / p)
    ret = 0
    for k in range(int_x+1):
        v1 = 1 - k * p
        tmp = v1**n
        
        v2 = 1 - (k + 1) * p
        if v2 >= 0:
            tmp -= v2**n
        
        ret += ((-1)**k) * C(n,k) * tmp
    return ret

def fast_duck_circle(n, degree, pp=True):
    ans = 1 - fast_no_sol_prob_scaled(n-1, Fraction(360-degree,360))
    if pp:
        print("the probability of %d ducks being within degree %d is %s, %s" % (n, degree, ans, float(ans)))
    return n, degree, ans, float(ans)

for i in range(1, 360 // 5):
    print(fast_duck_circle(4, i * step, False))
    
for i in range(1, 360 // 5):
    print(fast_duck_circle(3, i * step, False))

(4, 5, Fraction(1, 93312), 1.071673525377229e-05)
(4, 10, Fraction(1, 11664), 8.573388203017832e-05)
(4, 15, Fraction(1, 3456), 0.00028935185185185184)
(4, 20, Fraction(1, 1458), 0.0006858710562414266)
(4, 25, Fraction(125, 93312), 0.0013395919067215364)
(4, 30, Fraction(1, 432), 0.0023148148148148147)
(4, 35, Fraction(343, 93312), 0.0036758401920438956)
(4, 40, Fraction(4, 729), 0.0054869684499314125)
(4, 45, Fraction(1, 128), 0.0078125)
(4, 50, Fraction(125, 11664), 0.010716735253772291)
(4, 55, Fraction(1331, 93312), 0.014263974622770919)
(4, 60, Fraction(1, 54), 0.018518518518518517)
(4, 65, Fraction(2197, 93312), 0.023544667352537723)
(4, 70, Fraction(343, 11664), 0.029406721536351165)
(4, 75, Fraction(125, 3456), 0.03616898148148148)
(4, 80, Fraction(32, 729), 0.0438957475994513)
(4, 85, Fraction(4913, 93312), 0.052651320301783266)
(4, 90, Fraction(1, 16), 0.0625)
(4, 95, Fraction(6859, 93312), 0.07350608710562415)
(4, 100, Fraction(125, 1458), 0.08573388203017833)
(4, 105, Fract

In [21]:
int(Fraction(10,6))

1